<a href="https://colab.research.google.com/github/taitsmp/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/notebooks/shotgun_classification_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install ultralytics==8.0.77
%pip install echo1-coco-split
%pip install git+https://github.com/taitsmp/coco-to-yolo.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 kB 17.8 MB/s eta 0:00:00
  Cloning https://github.com/taitsmp/coco-to-yolo.git to /tmp/pip-req-build-019lsmrh
  Running command git clone --filter=blob:none --quiet https://github.com/taitsmp/coco-to-yolo.git /tmp/pip-req-build-019lsmrh
  Resolved https://github.com/taitsmp/coco-to-yolo.git to commit 4db16cc0886b40f18fdf5f58b748d4f0b56e69f8
  Preparing metadata (setup.py) ... done
  Created wheel for coco-to-yolo: filename=coco_to_yolo-0.1.5-py3-none-any.whl size=17608 sha256=d683668bd715797913c53aa3bd4e7f5feb563ae19e65ef75fb1165f91dacc804
  Stored in directory: /tmp/pip-ephem-wheel-cache-ahx9pxsy/wheels/0a/64/ca/05b497084c9a4182506c7c844249fd374a032633a71a182e19
Successfully built coco-to-yolo


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
import os
#os.environ['WANDB_MODE'] = 'disabled'

In [4]:
import wandb
from google.colab import userdata

wandb_key = userdata.get('wandb_key') #google colab secrets
wandb.login(key=wandb_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: telarson (fb-clips). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Variables

In [6]:
short_name = 'shotgun'
project_name = 'project-11' #internal project name related to label studio


epochs = 200
batch_size = 16
coco_file = "shotgun.20241223_071107.coco.json"
img_sz = 448
patience=40
use_weighted_classes = True #experimental. Review and make sure it does not override augmentation_params, etc.

mlops_project_name = f'{short_name}-classification' #used by wandb. Might be used by mlflow
model_arch = 'yolov8m-cls'
model_name = f'{short_name}_{model_arch}'

base_drive_dir = '/content/drive/Othercomputers/My Mac/fb-clips-data-science'
model_root_path = f"{base_drive_dir}/models"
image_root_path = f"/content/{project_name}/football-videos-pre-snap"

COCO_FILE = f'{base_drive_dir}/annotations/{coco_file}'
DATASET_DIR = '/content/dataset'
IMAGE_ROOT = '/content/dataset/images'

# organization of training data should be...
'''
/content/dataset/
├── images/
├── train.json
└── val.json
'''

YOLO_DATASET_DIR = '/content/yolo_dataset'
YOLO_IMAGE_ROOT = f'{YOLO_DATASET_DIR}/images'

# we then move things into a yolo-friendly dataset directory using coco-to-yolo
'''
/content/yolo_dataset
├── images/
│   ├── train/      # Training images
│   └── val/        # Validation images
└── labels/
    ├── train/      # Training labels (*.txt files)
    └── val/        # Validation labels (*.txt files)
'''


'\n/content/yolo_dataset\n├── images/\n│   ├── train/      # Training images\n│   └── val/        # Validation images\n└── labels/\n    ├── train/      # Training labels (*.txt files)\n    └── val/        # Validation labels (*.txt files)\n'

## Copy Images From Remote Storage

In [7]:
## Copy the data
# Install rclone
!curl -s https://rclone.org/install.sh | sudo bash > /dev/null

# Setup rclone config
!mkdir -p ~/.config/rclone
!cp "/content/drive/Othercomputers/My Mac/fb-clips-data-science/config/rclone.conf" ~/.config/rclone/

# Create project directory
!mkdir -p {IMAGE_ROOT}

# Copy data from remote storage
!rclone copy r2-ml:fb-clips-ml/{project_name} {IMAGE_ROOT} \
    --checkers=128 \
    --transfers=128 \
    --fast-list \
    --buffer-size=16M \
    --bwlimit=off \
    --log-level=ERROR


Updating index cache for path `/usr/share/man/man1'. Wait...done.
mandb: can't update index cache /var/cache/man/oldlocal/1230: No such file or directory
Updating index cache for path `/usr/local/man/man1'. Wait...done.


In [8]:
!find {IMAGE_ROOT} -type f | wc -l

11414


## Split the Data

In [9]:
%cd {DATASET_DIR}
!coco-split --annotations_file "{COCO_FILE}" --valid_name val.json --valid_ratio 0.2 --test_ratio 0.0
%cd -

/content/dataset
Saved 4707 annotations from 4707 images to train.json
Saved 1177 annotations from 1177 images to val.json
/content


In [10]:
!coco_to_yolo {DATASET_DIR} {YOLO_DATASET_DIR} --images-dir {IMAGE_ROOT} --classification


Input directory: /content/dataset
Output directory: /content/yolo_dataset
Images directory: /content/dataset/images
Mode: Classification

Found split files: ['train', 'val']

Processing train split for classification...
Split: train
  Total images in annotations: 4707
  Successfully processed: 4707

Processing val split for classification...
Split: val
  Total images in annotations: 1177
  Successfully processed: 1177

Conversion complete!
Dataset created at: /content/yolo_dataset

Final Class Mapping Summary:
---------------------------
YOLO ID | Original ID | Class Name
---------------------------
      0 |         19 | no
      1 |         24 | yes
---------------------------


## Use Weight Classes (Optional)

In [11]:
# weighted_dataset.py

if use_weighted_classes:
    import numpy as np
    from ultralytics.yolo.data.dataset import ClassificationDataset
    from ultralytics.yolo.data import build

    class WeightedClassificationDataset(ClassificationDataset):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)

            # Only apply weighting during training
            self.train_mode = "train" in str(self.root).lower()
            if self.train_mode:
                self.setup_weighting()

        def setup_weighting(self):
            """Set up class weights for balanced sampling during training."""
            # Count instances per class
            class_counts = np.zeros(len(self.classes))
            for _, class_idx, _, _ in self.samples:
                class_counts[class_idx] += 1

            # Prevent division by zero by setting min count to 1
            class_counts = np.maximum(class_counts, 1)

            # Calculate inverse weights (fewer samples = higher weight)
            self.class_weights = np.sum(class_counts) / (len(self.classes) * class_counts)

            # Calculate sample weights based on their class
            self.sample_weights = np.array([self.class_weights[idx] for _, idx, _, _ in self.samples])

            # Normalize probabilities
            self.probabilities = self.sample_weights / np.sum(self.sample_weights)

        def __getitem__(self, index):
            """Get item with weighted sampling during training."""
            if self.train_mode:
                # Use weighted sampling only during training
                index = np.random.choice(len(self.samples), p=self.probabilities)

            # Use parent class to handle all the augmentation and processing
            return super().__getitem__(index)

    # Apply the patch
    build.ClassificationDataset = WeightedClassificationDataset
    print("using weigheted classes")
else:
    print("weighted classes turned off")

using weigheted classes


## Configure Training

In [12]:
augmentation_params = {
        'hsv_h': 0.4,       # Strong hue shift
        'hsv_s': 0.6,       # Strong saturation augmentation
        'hsv_v': 0.4,       # Moderate value augmentation
        'degrees': 3.0,     # Moderate rotation
        'translate': 0.1,   # Moderate translation
        'scale': 0.1,       # Moderate scaling
        'fliplr': 0.45,      # Horizontal flip okay
        'mosaic': 0.0,      # No mosaic
        'mixup': 0.0,       # No mixup
        'copy_paste': 0.0,  # No copy-paste
        'perspective': 0.0007, # Slight perspective change
    }

In [13]:
import json

# count the training examples
with open(f'{DATASET_DIR}/train.json') as f:
    tji = json.load(f)
    tss = len(tji['images'])


## Train

In [ ]:
print(img_sz)

736


In [14]:
from ultralytics import YOLO

model = YOLO(f'{model_arch}.pt')

results = model.train(
    data=f'{YOLO_DATASET_DIR}',
    epochs=epochs,
    imgsz=img_sz,
    batch=batch_size,
    **augmentation_params,
    task="classify",
    name=model_name + ("_balanced" if use_weighted_classes else ""),
    #wandb params below
    project=mlops_project_name,
    save_period=12,
    patience=patience,
    resume=False # you probably don't want to resume a run b/c training is fast. change this if you do.
)

#trying to save some extra params here.
wandb.config.update({
    'weighted_training': use_weighted_classes,
    'coco_file': coco_file,
    'training_set_size': tss,
})

# explicitly finish
wandb.finish()


100%|██████████| 32.7M/32.7M [00:00<00:00, 57.3MB/s]
/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  retur

AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
100%|██████████| 6.23M/6.23M [00:00<00:00, 112MB/s]
/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on 

lr/pg0,█████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
lr/pg1,█████▇▆▆▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
lr/pg2,███▇▇▇▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
metrics/accuracy_top1,▁▃▁▂▂▃▃▅▅▄▆▅▆▅▆▆▆▇▇▆▇▆▆▇▇▇▇▇▇▇██████████
metrics/accuracy_top5,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model/GFLOPs,▁
model/parameters,▁
model/speed(ms),▁
train/loss,███████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▃▂▂▂▁
val/loss,██▇▇█▇▇▆▆▆▅▅▅▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
lr/pg0,0.0002


In [ ]:
#run this if you cancelled

if wandb.run:
      #trying to save some extra params here.
    wandb.config.update({
        'weighted_training': use_weighted_classes,
        'coco_file': coco_file,
        'training_set_size': tss,
    })

    # explicitly finish
    wandb.finish()


lr/pg0,█▅▁▁▁
lr/pg1,▁▅███
lr/pg2,▁▅███
metrics/accuracy_top1,▁█▅▆
metrics/accuracy_top5,▁▁▁▁
model/GFLOPs,▁
model/parameters,▁
model/speed(ms),▁
train/loss,██▁▅▁
val/loss,▅▂█▁
lr/pg0,0.00985


## Save the Model

In [15]:
import re
from datetime import datetime

date_match = re.search(r'at-(\d{4}-\d{2}-\d{2})', coco_file)
date_mtch2 = re.search(r'-(\d{4}\d{2}\d{2})_(\d+)', coco_file)
if date_match:
    # Convert the matched date string to datetime and reformat
    date_suf = datetime.strptime(date_match.group(1), '%Y-%m-%d').strftime('%Y%m%d')
    model_fullname = f"{model_name}-{date_suf}.pt"
elif date_mtch2:
    # Convert the matched date string to datetime and reformat
    date_suf = date_mtch2.group(1)
    extra_suf = date_mtch2.group(2)
    model_fullname = f"{model_name}-{date_suf}_{extra_suf}.pt"
else:
    # Fallback to current date if pattern not found
    date_suf = datetime.now().strftime('%Y%m%d')
    model_fullname = f"{model_name}-{date_suf}.pt"

best_model_path = str(model.trainer.best)

In [16]:
best_model_path

'shotgun-classification/shotgun_yolov8m-cls_balanced/weights/best.pt'

In [17]:
import shutil
import os

def copy_with_versioning(source_path, dest_path):
    """
    Copy a file to the destination path. If a file with the same name exists,
    append a version number (e.g., .2, .3) before the file extension.
    """
    base, ext = os.path.splitext(dest_path)
    version = 1

    while os.path.exists(dest_path):
        version += 1
        if version > 9:
            raise Exception("Exceeded single digit version limits")
        dest_path = f"{base}.{version}{ext}"

    shutil.copy(source_path, dest_path)
    return dest_path


# Example usage
source_path = best_model_path  # Path to the source file
dest_path = f"{model_root_path}/{model_fullname}"  # Destination path

gm_loc = copy_with_versioning(source_path, dest_path)

In [18]:
print(f'copied model to f{gm_loc}')

copied model to f/content/drive/Othercomputers/My Mac/fb-clips-data-science/models/shotgun_yolov8m-cls-20241223.2.pt


## Visualize Results

In [ ]:
import os
import re
from pathlib import Path

def get_last_run_folder(base_path=f'/content/{mlops_project_name}/{model_name}'):
    # Get the parent directory of the provided path
    parent_dir = os.path.dirname(base_path)

    # Extract the base pattern from the provided base_path
    base_pattern = os.path.basename(base_path)
    base_regex = re.escape(base_pattern) + r'\d*$'

    # Get all subdirectories in the parent directory
    folders = [
        d for d in os.listdir(parent_dir)
        if os.path.isdir(os.path.join(parent_dir, d)) and re.match(base_regex, d)
    ]

    if folders:
        # Sort numerically by trailing numbers and take the last one
        return sorted(
            folders,
            key=lambda x: int(re.findall(r'\d+$', x)[0]) if re.findall(r'\d+$', x) else 0
        )[-1]

    # If no folders found, return None
    return None

train_folder = get_last_run_folder()

In [ ]:
# TODO: remove me.
train_folder

'snap_yolov8m-cls'

In [ ]:
import base64
from IPython.display import Image, display, HTML


def embed_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return f"""
    <div>
        <img
            src="data:image/jpeg;base64,{encoded_string}"
            style="max-width: 800px"
        />
    </div>
    """

In [ ]:
HTML(embed_image(f'/content/{mlops_project_name}/{train_folder}/confusion_matrix.png'))